In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()

In [3]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 10B6-0DC8

 Directory of C:\Users\kanti\OneDrive\Desktop\ML session\Pyspark

07/04/2023  20:06    <DIR>          .
06/04/2023  20:00    <DIR>          ..
07/04/2023  20:06    <DIR>          .ipynb_checkpoints
04/04/2023  20:08                23 .jovianrc
17/09/2022  07:08       140.600.832 city_temperature.csv
14/07/2017  22:32             8.734 cruise_ship_info.csv
06/04/2023  20:43                57 dept.csv
06/04/2023  20:42                61 emp.csv
07/04/2023  20:06               589 Pyspark - ML.ipynb
06/04/2023  21:01            50.965 pyspark sql.ipynb
04/04/2023  20:59            41.389 Pyspark2.ipynb
05/04/2023  21:04            32.194 Pyspark3.ipynb
04/04/2023  20:44                77 test.csv
05/04/2023  20:37               150 test12.csv
              11 File(s)    140.735.071 bytes
               3 Dir(s)  38.553.800.704 bytes free


In [65]:
df = spark.read.csv("/FileStore/tables/tset1/cruise_ship_info.csv",inferSchema=True,header=True)

AnalysisException: Path does not exist: file:/FileStore/tables/tset1/cruise_ship_info.csv

In [6]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [7]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     null|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [17]:
df.groupBy('Cruise_line').count().count()

20

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
encoded = StringIndexer(inputCol="Cruise_line",outputCol="encoded_Cruise_line")

In [12]:
encoded_fit = encoded.fit(df).transform(df)

In [14]:
encoded_fit.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|encoded_Cruise_line|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [29]:
encoded_fit.groupBy('encoded_Cruise_line').max().show()

+-------------------+--------+------------------+---------------+-----------+-----------+----------------------+---------+------------------------+
|encoded_Cruise_line|max(Age)|      max(Tonnage)|max(passengers)|max(length)|max(cabins)|max(passenger_density)|max(crew)|max(encoded_Cruise_line)|
+-------------------+--------+------------------+---------------+-----------+-----------+----------------------+---------+------------------------+
|                8.0|      29|             115.0|          35.74|       9.35|      15.32|                 43.19|     12.2|                     8.0|
|                0.0|      25|             220.0|           54.0|      11.82|       27.0|                 44.32|     21.0|                     0.0|
|                7.0|      36|             133.5|          39.59|      10.93|      16.37|                 37.71|    13.13|                     7.0|
|               18.0|      15|            83.338|           17.5|       9.64|       8.75|                 47.62|

In [25]:
import numpy as np


np.max(encoded_fit.select('encoded_Cruise_line'))

DataFrame[encoded_Cruise_line: double]

In [34]:
encoded_fit.select('encoded_Cruise_line').describe().show()

+-------+-------------------+
|summary|encoded_Cruise_line|
+-------+-------------------+
|  count|                158|
|   mean|  5.063291139240507|
| stddev|  4.758744608182735|
|    min|                0.0|
|    max|               19.0|
+-------+-------------------+



In [35]:
from pyspark.ml.feature import VectorAssembler

In [37]:
encoded_fit.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|encoded_Cruise_line|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [38]:
encoded_fit.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'encoded_Cruise_line']

In [41]:
assembler = VectorAssembler(inputCols=[
  'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'encoded_Cruise_line'],outputCol = 'features')

In [42]:
output = assembler.transform(encoded_fit)

In [43]:
output.select("features","crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [44]:
final_data = output.select("features","crew")

In [45]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [47]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [49]:
train_data.count()

110

In [50]:
(158/100)*70

110.60000000000001

In [51]:
from pyspark.ml.regression import LinearRegression

In [52]:
lg = LinearRegression(labelCol='crew')

In [53]:
model = lg.fit(train_data)

In [55]:
r = model.evaluate(test_data)

In [57]:
r.predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[6.0,30.276999999...| 3.55| 4.607342599468423|
|[6.0,30.276999999...| 3.55| 4.607342599468423|
|[6.0,90.0,20.0,9....|  9.0|    10.19750413928|
|[7.0,116.0,31.0,9...| 12.0|12.469471424538337|
|[7.0,158.0,43.7,1...| 13.6|13.689193204818409|
|[8.0,77.499,19.5,...|  9.0| 8.523192577301835|
|[9.0,90.09,25.01,...| 8.69| 9.167579061315736|
|[9.0,105.0,27.2,8...|10.68|11.161985560859197|
|[9.0,113.0,26.74,...|12.38|11.213223972215632|
|[10.0,46.0,7.0,6....| 4.47|2.9656138657821605|
|[10.0,58.825,15.6...|  7.0| 7.336676055947184|
|[10.0,77.0,20.16,...|  9.0| 8.803483694488296|
|[10.0,90.09,25.01...| 8.58| 8.794508993602973|
|[11.0,110.0,29.74...| 19.1|11.873532536780967|
|[12.0,2.329,0.94,...|  0.6|  0.77660696594947|
|[12.0,77.104,20.0...| 9.59| 8.697089115304832|
|[12.0,90.09,25.01...| 8.68| 8.750080663182443|
|[12.0,108.865,27....| 11.0| 10.69354494

In [58]:
r.rootMeanSquaredError

1.3695502198643417

In [59]:
r.meanSquaredError

1.875667804730467

In [60]:
r.r2

0.8523739250478218

In [61]:
r.r2adj

0.8265393619311907

In [64]:
from pyspark.sql.functions import corr

df.select(corr("crew",'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [ ]:
fit(x_train,y_train)  -- all -- collective 

In [67]:
!pip install databricks

In [70]:
from databricks import sql
import os

with sql.connect(server_hostname = "community.cloud.databricks.com",
                 http_path       = "sql/protocolv1/o/1728881905583399/0407-152221-xd0li0c1",
                 access_token    = "1728881905583399") as connection:

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM default.diamonds LIMIT 2")
    result = cursor.fetchall()
    for row in result:    print(row)

IndentationError: expected an indented block (1169329248.py, line 8)